In [1]:
"""
https://wiki.geneontology.org/index.php/Guide_to_GO_Evidence_Codes

TODO
add arg parser
transform into a script
add more evidence codes
use GOA parser?
"""

import pandas as pd
import numpy as np
import networkx
# conda install -c biobuilds obonet
import obonet

In [2]:
data_dir = "/home/damiano/Projects/CAFA-evaluator_data"

# The OBO must have "ontology: IDPO" header (first line)
graph = obonet.read_obo("{}/go_21_oct_2022.obo".format(data_dir))
# graph.nodes(data=True)
df_ont = pd.DataFrame([[node[0], node[1]['namespace'], node[1]['name']] for node in graph.nodes(data=True)], columns=['term', 'namespace', 'name'])
df_ont

,term,namespace,name
0,GO:0000001,biological_process,mitochondrion inheritance
1,GO:0000002,biological_process,mitochondrial genome maintenance
2,GO:0000003,biological_process,reproduction
3,GO:0000006,molecular_function,high-affinity zinc transmembrane transporter a...
4,GO:0000007,molecular_function,low-affinity zinc ion transmembrane transporte...
...,...,...,...
43324,GO:2001314,biological_process,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...
43325,GO:2001315,biological_process,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...
43326,GO:2001316,biological_process,kojic acid metabolic process
43327,GO:2001317,biological_process,kojic acid biosynthetic process


In [3]:
# Create the ancestors dictionary
ancestors_dict = {}
for node in graph.nodes(data=True):
    # print(node[0], networkx.descendants(graph, node[0]), node[1].get('is_a'))
    ancestors_dict[node[0]] = networkx.descendants(graph, node[0])
ancestors_dict

{'GO:0000001': {'GO:0006996',
  'GO:0007005',
  'GO:0008150',
  'GO:0009987',
  'GO:0016043',
  'GO:0048308',
  'GO:0048311',
  'GO:0051179',
  'GO:0051640',
  'GO:0051646',
  'GO:0071840'},
 'GO:0000002': {'GO:0006996',
  'GO:0007005',
  'GO:0008150',
  'GO:0009987',
  'GO:0016043',
  'GO:0071840'},
 'GO:0000003': {'GO:0008150'},
 'GO:0000006': {'GO:0000041',
  'GO:0003674',
  'GO:0005215',
  'GO:0005385',
  'GO:0006810',
  'GO:0006811',
  'GO:0006812',
  'GO:0006829',
  'GO:0008150',
  'GO:0008324',
  'GO:0009987',
  'GO:0015075',
  'GO:0015318',
  'GO:0022857',
  'GO:0022890',
  'GO:0030001',
  'GO:0034220',
  'GO:0046873',
  'GO:0046915',
  'GO:0051179',
  'GO:0051234',
  'GO:0055085',
  'GO:0071577',
  'GO:0098655',
  'GO:0098660',
  'GO:0098662'},
 'GO:0000007': {'GO:0000041',
  'GO:0003674',
  'GO:0005215',
  'GO:0005385',
  'GO:0006810',
  'GO:0006811',
  'GO:0006812',
  'GO:0006829',
  'GO:0008150',
  'GO:0008324',
  'GO:0009987',
  'GO:0015075',
  'GO:0015318',
  'GO:0022857'

In [4]:
# Parse Swiss-Prot annotations, filter by evidence codes and add ancestors
# EXP, IDA, IMP, IGI, IEP, TAS, IC
eco_mapping = [('EXP', 'ECO:0000269'), ('IDA', 'ECO:0000314'), ('IPI', 'ECO:0000353'), ('IMP', 'ECO:0000315'),
               ('IGI', 'ECO:0000316'), ('IEP', 'ECO:0000270'), ('HTP', 'ECO:0006056'), ('HDA', 'ECO:0007005'),
               ('HMP', 'ECO:0007001'), ('HGI', 'ECO:0007003'), ('HEP', 'ECO:0007007'), ('ISS', 'ECO:0000250'),
               ('ISO', 'ECO:0000266'), ('ISA', 'ECO:0000247'), ('ISM', 'ECO:0000255'), ('IGC', 'ECO:0000317'),
               ('IBA', 'ECO:0000318'), ('IBD', 'ECO:0000319'), ('IKR', 'ECO:0000320'), ('IRD', 'ECO:0000321')]
valid_eco = set([eco for ec, eco in eco_mapping if ec in ['EXP', 'IDA', 'IMP', 'IGI', 'IEP', 'TAS', 'IC']])

reference_file = "{}/uniprot_sprot_go.tsv".format(data_dir)

reference_dataset = {}
with open(reference_file) as f:
    for line in f:
        acc, term, eco = line.strip().split()
        if eco in valid_eco and term in ancestors_dict:
            reference_dataset.setdefault(acc, set()).add(term)
            reference_dataset[acc].update(ancestors_dict[term])

In [5]:
len(reference_dataset)

68628

In [7]:
ia = {}  # {term: [observed ancestors]
for i, acc in enumerate(reference_dataset):
    if i % 1000 == 0:
        print(i)
    for term in df_ont['term']:
        parents = set(graph.successors(term))
        if parents.issubset(reference_dataset[acc]):
            ia.setdefault(term, [0, 0])
            ia[term][0] += 1
        parents.add(term)
        if parents.issubset(reference_dataset[acc]):
            ia.setdefault(term, [0, 0])
            ia[term][1] += 1
len(ia)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000


37586

In [ ]:
df_prob = pd.DataFrame([[k, *v] for k, v in ia.items()], columns=['term', 'co_occurring_parents', 'co_occurring'])
df_prob['p_cond'] = df_prob['co_occurring'] / df_prob['co_occurring_parents']
df_prob['ia'] = -np.log2(df_prob['p_cond'])
df_prob.to_csv('{}/information_content.tsv'.format(data_dir), sep='\t', index=False)
df_prob